In [30]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import alpha_vantage as av
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.techindicators import TechIndicators as ti
from alpha_vantage.timeseries import TimeSeries as ts
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas

In [31]:
load_dotenv()

True

In [32]:
# Specify paper trading environment & set API keys
alpaca_paper_endpoint=os.getenv("ALPACA_PAPER_ENDPOINT")
alpaca_api_key=os.getenv("ALPACA_API_KEY")
alpaca_secret_key=os.getenv("ALPACA_SECRET_KEY")
# Create alpaca API object
alpaca_api=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    alpaca_paper_endpoint,
    api_version='v2'
)
# List Positions
positions=alpaca_api.list_positions()
# Put positions and extra info into DataFrame
positions_df=pd.DataFrame({
    'exchange':[x.exchange for x in positions],
    'ticker':[x.symbol for x in positions],
    'qty':[x.qty for x in positions],
    'lastday_price':[x.lastday_price for x in positions],
    'current_price':[x.current_price for x in positions],
    'change_today':[x.change_today for x in positions],
    'market_value':[x.market_value for x in positions],
    'unrealized_intraday_pl':[x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc':[x.unrealized_intraday_plpc for x in positions],
})
# Create tickers list from positions_df DataFrame
# positions_tickers=positions_df.ticker.tolist()
positions_tickers=['RCL','CPE']

In [34]:
# Setup AV API
av_api_key=os.getenv("ALPHAVANTAGE_API_KEY")
av_api='https://www.alphavantage.co/query?'
# Create technical indicators query
ti_query=ti(key=av_api_key,output_format='pandas')
# Create time-series query
ts_query=ts(key='av_api_key',output_format='pandas')
# Set query input variables
interval='60min'
period=60
outputsize='full'
series_type='close'

In [35]:
# Set sleep interval (for testing with less indicators and tickers)
sleep=13

In [36]:
# Loop through tickers to retrieve plot data
tabs=pn.Tabs()
for x in positions_tickers:
    
    # Get RSI
    print(f"Getting RSI for {x} ticker")
    rsi_df,meta_data_rsi=ti_query.get_rsi(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get SMA
    print(f"Getting SMA for {x} ticker")
    sma_df,meta_data_sma=ti_query.get_sma(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get EMA
    print(f"Getting EMA for {x} ticker")
    ema_df,meta_data_ema=ti_query.get_ema(symbol=x,
                                          interval=interval,
                                          time_period=period,
                                          series_type=series_type,
    )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)
    
#     # Get OBV
#     print(f"Getting OBV for {x} ticker")
#     obv_df,meta_data_obv=ti_query.get_obv(symbol=x,
#                                           interval=interval,
#     )    
#     # Sleep
#     print(f"Sleeping for {sleep} seconds...")
#     time.sleep(sleep)
    
    # Get BBANDS
    print(f"Getting BBANDS for {x} ticker")
    bbands_df,meta_data_bbands=ti_query.get_bbands(symbol=x,
                                                   interval=interval,
                                                   time_period=period,
                                                   series_type=series_type,
    )
    # Rename BBANDS columns
    bbands_df=bbands_df.rename(columns={'Real Upper Band':'Upper Band',
                                        'Real Lower Band':'Lower Band',
                                        'Real Middle Band':'Middle Band'}
                              )
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep) 

    # Get MACD
    print(f"Getting MACD for {x} ticker")
    macd_df,meta_data_macd=ti_query.get_macd(symbol=x,
                                             interval=interval,
                                             series_type=series_type,
    )
    #Drop Extra MACD columns returned
    macd_df=macd_df.drop(['MACD_Signal','MACD_Hist'],axis=1)
    # Sleep
    print(f"Sleeping for {sleep} seconds...")
    time.sleep(sleep)

    # Get Time-Series
    print(f"Getting time-series for {x} ticker")
    data_ts,meta_data_ts=ts_query.get_intraday(symbol=x,
                                               interval=interval,
                                               outputsize=outputsize,
    )
    # Create time-series DataFrame
    print(f"Creating time-series DataFrame for {x} ticker")
    ts_df=pd.DataFrame.from_dict(data_ts)
    ts_df=ts_df.drop(['5. volume','3. low','2. high','1. open'],axis=1)
    ts_df=ts_df.rename(columns={'4. close':'Closing Price'})
    
    # Merge all DataFrames
    ticker_indicator_df=pd.concat([bbands_df,ema_df,rsi_df,sma_df,macd_df,ts_df],axis='columns',join='inner')
# obv_df,
    for t in x:
        # Create plot
        print(f"Creating plot for {x} ticker...")
        ticker_indicator_plot=ticker_indicator_df.hvplot(
                                                         kind='line',
                                                         x='date',
                                                         xlabel='Date',
                                                         ylabel='Values',
                                                         legend=True,
                                                         rot=90,
                                                         group_label='Indicator',
                                                         title=f'Combined Indicator Plot for {x} Symbol',
                                                         width=1000,
                                                         height=500, 
        )

        # Create Panel for plot
    ticker_panel=pn.Column(f'{x} Plot',ticker_indicator_plot)

    # Put the panel in a tab
    tabs.append((f'Indicator Chart for {x}',ticker_panel),)
    # Sleep
    print(f"Sleeping for {sleep} seconds before moving to next ticker...")
    time.sleep(sleep)   

# Serve the dashboard
pn.serve(title='Technical Indicator Plots',panels=tabs)
#          ({[x.tab for x in positions_tickers]}))

print('Complete!')

Getting RSI for RCL ticker
Sleeping for 13 seconds...
Getting SMA for RCL ticker
Sleeping for 13 seconds...
Getting EMA for RCL ticker
Sleeping for 13 seconds...
Getting BBANDS for RCL ticker
Sleeping for 13 seconds...
Getting MACD for RCL ticker
Sleeping for 13 seconds...
Getting time-series for RCL ticker
Creating time-series DataFrame for RCL ticker
Creating plot for RCL ticker...
Creating plot for RCL ticker...
Creating plot for RCL ticker...
Sleeping for 13 seconds before moving to next ticker...
Getting RSI for CPE ticker
Sleeping for 13 seconds...
Getting SMA for CPE ticker
Sleeping for 13 seconds...
Getting EMA for CPE ticker
Sleeping for 13 seconds...
Getting BBANDS for CPE ticker
Sleeping for 13 seconds...
Getting MACD for CPE ticker
Sleeping for 13 seconds...
Getting time-series for CPE ticker
Creating time-series DataFrame for CPE ticker
Creating plot for CPE ticker...
Creating plot for CPE ticker...
Creating plot for CPE ticker...
Sleeping for 13 seconds before moving to n